<a href="https://www.kaggle.com/code/eneserenseven/pytorch-mnist?scriptVersionId=264098483" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets 
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")        
def get_data_loader(BATCH_SIZE = 64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,),(0.5,))
    ])
    train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    test_loader   = torch.utils.data.DataLoader(test_set,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    return train_loader, test_loader

get_data_loader()

train_loader, test_loader = get_data_loader()

def visualize_samples(loader,n):
    images, labels = next(iter(loader))
    fig, axes = plt.subplots(1,n, figsize=(10,5))
    for i in range(n):
        axes[i].imshow(images[i].squeeze(), cmap ="gray")
        axes[i].set_title(f"Label : {labels[i].item()}")
        axes[i].axis("off")
    plt.show

visualize_samples(train_loader,4)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        
model = NeuralNetwork().to(device)

define_loss_and_optimizer = lambda model:(
    nn.CrossEntropyLoss(),
    optim.Adam(model.parameters(),lr = 0.001)
)
criterion, optimizers = define_loss_and_optimizer(model)

def train_model(model, train_loader,criterion, optimizer, epochs = 10):
    model.train()
    train_losses = []
    for epoch in range(epochs):
        total_loss= 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            predictions = model(images)
            loss = criterion(predictions,labels)
            loss.backward()
            optimizers.step()
            total_loss = total_loss + loss.item()

        avg_loss = total_loss / len(train_loader)
        train_losses.append(avg_loss)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.3f}")

    plt.figure()
    plt.plot(range(1,epochs+1), train_losses, marker = "o", linestyle = "-", label = "train" )
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training Loss")
    plt.legend()
    plt.show
    
def test_model(model,test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images,labels = images.to(device), labels.to(device)
            predictions = model(images)
            _, predicted = torch.max(predictions, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test: Accuracy: {100*correct/total:.3f}%")

if __name__ == "__main__":
    train_loader, test_loader = get_data_loader()
    visualize_samples(train_loader,5)
    criterion,optimizer = define_loss_and_optimizer(model)
    train_model(model,train_loader,criterion, optimizer)
    test_model(model,test_loader)